In [3]:
import pandas as pd
import numpy as np
import ast

In [4]:
df = pd.read_csv("./experiments/os_datasets.csv")

In [102]:
(df[["number"]].value_counts() == 2).sum()

85

In [5]:
df = df.drop_duplicates(subset=["number"])

In [104]:
(df[["number"]].value_counts() == 2).sum()

0

In [6]:
def flatten_list(array):
    response = [item for sublist in array for item in sublist]
    return response

### Extending the csv file with single-component related changes that do not appear in other co-changes

In [7]:
result_number_changes = pd.read_csv("./experiments/all_paths.csv")

In [8]:
result_number_changes = result_number_changes["Path"].apply(ast.literal_eval).values.tolist()

In [129]:
result_number_co_changes = result_number_changes["Path"].copy().to_list()

In [133]:
result_number_co_changes = [item.tolist() for item in result_number_co_changes]

In [ ]:
result_number_co_changes

In [135]:
all_paths_flattend = list(dict.fromkeys(flatten_list(result_number_co_changes)))

In [145]:
single_component_changes_number = df.loc[~df["number"].isin(all_paths_flattend), ["number"]].values.tolist()

In [146]:
extended_paths_number = result_number_co_changes + single_component_changes_number

In [147]:
print("result_number_co_changes %d"%len(result_number_co_changes))
print("single_component_changes_number %d"%len(single_component_changes_number))
print("extended_paths_number %d"%len(extended_paths_number))

result_number_co_changes 215309
single_component_changes_number 214328
extended_paths_number 429637


### Replace numbers with repos in extended_paths_number 

In [159]:
def number_to_repo(data, df):
    df_subset = df[["number", "project"]]
    result = []
    for i in range(len(data)):
        row = data[i]
        new_row = df_subset.loc[
            df_subset["number"].isin(row), ["project"]].drop_duplicates(
                subset="project").values.reshape(-1).tolist()
        result.append(new_row)
    return result

In [149]:
def combine_numbers(data):
    result = []
    data = data.copy()
    for i in range(len(data)):
        new_item = {}

        if len(data[i]) == 0:
            continue

        for j in range(1, len(data)):
            if len(data[j]) == 0 or (data[i] == data[j]):
                continue

            check_any = any(item in data[i] for item in data[j])
            if check_any == True:
                new_item = new_item | dict.fromkeys(data[i] + data[j])
                data[j] = []
        
        if len(new_item) == 0:
            result.append(data[i])
        else:
            result.append(list(new_item))
        data[i] = []

    return result

In [152]:
test = extended_paths_number[:2000].copy()
test_combined_number = combine_numbers(test)

In [155]:
pd.DataFrame({"Path": test_combined_number}).to_csv("test_number.csv", index=False)

In [160]:
test_exteded_repo = number_to_repo(test_combined_number, df)

In [162]:
pd.DataFrame({"Path": test_exteded_repo}).to_csv("test_repo.csv", index=False)

In [164]:
numbers = [865314, 865315, 212127, 599335, 579277, 510663, 485956, 465150, 461122, 360999, 360991, 96492, 688546, 688548, 722173, 722174, 722175, 722177, 722176, 689588, 687978, 269262, 362709, 182056, 109582, 109639, 99726, 711275]

In [165]:
len(numbers)

28

In [172]:
repos = df.loc[df["number"].isin(numbers), ["project"]].drop_duplicates(subset="project").values.reshape(-1).tolist()

In [ ]:
test = extended_paths_number[:10]

In [23]:
def remove_duplicates(data):
    for i in range(len(data)):
        data[i] = list(dict.fromkeys(data[i]))
    return data

In [24]:
df_extended_paths_repo = pd.read_csv("./Files/extended_paths.csv")

In [25]:
df_extended_paths_repo = df_extended_paths_repo["Path"].apply(ast.literal_eval).values.tolist()

In [26]:
df_extended_paths_repo = remove_duplicates(df_extended_paths_repo)

In [27]:
pd.DataFrame({"Path": df_extended_paths_repo}).to_csv("./Files/Openstack_Evolution_Clean_Paths.csv", index=False)